# Technical test for the data science internship @Hyperlex

In [1]:
import os
import json
import numpy as np
import pandas as pd
import nltk

In [2]:
!find . -iname *checkpoint* -type d -exec rm -r {} +  

FIND: Parameter format not correct


In [3]:
rootdir = 'dataset'
json_files = [os.path.join(dirpath, file) for dirpath,_,files in os.walk(rootdir) for file in files if file.endswith('json')]

data = []
for file in json_files:
    with open(file) as f:
        data.append(json.load(f)['clauses'][1:])  # starting from one to skip the summary since they contain different informations 
                                                  # and could affect the classification
articles = [article for contract in data for article in contract ]

In [4]:
articles_df = pd.DataFrame(articles, columns=['text'])

In [7]:
articles_df.describe()

,text
count,22782
unique,18992
top,"ARTICLE 10 FORME, CESSION ET TRANSMISSION DES ..."
freq,31


we can see that we have multiple duplicates. let's drop them to avoid adding a bias to the classifier

In [10]:
articles_df.drop_duplicates(inplace=True)
articles_df.describe()

,text
count,18992
unique,18992
top,ARTICLE 21. LIQUIDATION
freq,1


In [11]:
articles_df

,text
0,ARTICLE 1 Forme La société est une entreprise ...
1,ARTICLE 2 Objet La société a pour objet en Fra...
2,ARTICLE 3 Dénomination La dénomination de la s...
3,ARTICLE 4 Siège social Le siège social est fix...
4,ARTICLE 5 Durée La durée de la société est fix...
5,ARTICLE 6 Exercice social L exercice social co...
6,"ARTICLE 7 Apports CLOVIS, SARL au capital de 5..."
7,ARTICLE 9 Augmentation et réduction du capital...
8,ARTICLE 10 Représentation des parts sociales L...
9,ARTICLE 1 1 Cession et transmission des parts ...
